In [107]:
from binascii import hexlify
from binascii import unhexlify
from hashlib import sha256
from hashlib import sha512
import hmac
from ecc import PrivateKey, S256Point, Signature
from helper import encode_base58_checksum, decode_base58, decode_base58_extended
from io import BytesIO, StringIO

N = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141

In [108]:
bip39_output = 0x72f98262a11e83345546b291a7244b850bc1f69975510c33463dd087868cc8289cf3c1a76d6cb87fc8ee447c89eed756a9311a2414970d65bb62e0b3012c0583

In [109]:
master_priv_key = bip39_output.to_bytes(64,"big") [:32]
master_priv_key.hex()

'72f98262a11e83345546b291a7244b850bc1f69975510c33463dd087868cc828'

In [110]:
chain_code = bip39_output.to_bytes(64,"big")[32:]
chain_code.hex()

'9cf3c1a76d6cb87fc8ee447c89eed756a9311a2414970d65bb62e0b3012c0583'

In [111]:
def extended_privkey(privkey,chain_code, _index, testnet=False):
    if testnet:
        version= 0x04358394.to_bytes(4,"big")
    else:
        version= 0x0488ade4.to_bytes(4,"big")
    depth=0x00.to_bytes(1,"big")
    fingerprint =0x00.to_bytes(4,"big")
    index = _index.to_bytes(4,"big")
    return version+depth+fingerprint+index+chain_code+b'\x00'+privkey

def deserialize_xprvk(s):
    version = s.read(4)
    depth = s.read(1)
    fingerprint = s.read(4)
    index= s.read(4)
    chain_code = s.read(32)
    s.read(1)
    privkey = s.read(32)
    return [version,depth, fingerprint,index,chain_code,privkey]

xtended_private_key = extended_privkey(master_priv_key, chain_code, 0)
xtended_private_key = encode_base58_checksum(xtended_private_key)
xtended_private_key

'xprv9s21ZrQH143K3czKtJzXTjafD4G61LSYzUNx3PdAiMmfp9vbatMf6GXv9nuHfMy5Lgc8igBdEfDYGSHeub79KzPb2EVmH3Lin4tePwpXUjo'

In [112]:
xtprvk = 'xprv9s21ZrQH143K2E7weJFDreP9A8dFkx18swFoAJ6DBdsVuSPoFBLzekLca8vEG1i6ikCcvzZb79joaEZAjeM8Nz1DZ549aCnv3YFXy33mLoW'
decoded =deserialize_xprvk( BytesIO(decode_base58_extended(xtprvk)))
decoded

[b'\x04\x88\xad\xe4',
 b'\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x10\xe8\\\x06\x82o\xdc\xd0\x06\xc2D\xfflD \x00D\xc1A\x8b\xdd\x82\xcc\x08\x8c3\xe4\x8a\xe7\xb4\xc2\xca',
 b'\xf2\x14\xc0\xaf\xd9\xb3\x11TO\xdbhl\xaeQ\x00\xf1\xac\n\xc8\xab\xa2\xb6\xc5W\x19\xbaA\xac-O-\x12']

In [113]:
for x in decoded:
    print(hex(int.from_bytes(x,"big")))

0x488ade4
0x0
0x0
0x0
0x10e85c06826fdcd006c244ff6c44200044c1418bdd82cc088c33e48ae7b4c2ca
0xf214c0afd9b311544fdb686cae5100f1ac0ac8aba2b6c55719ba41ac2d4f2d12


In [114]:
master_priv_key = decoded[-1]
chain_code = decoded[-2]

In [115]:
def str_i(i):
    i_str = hex(i)[2:]
    while len(i_str)<8:
        i_str="0"+i_str
    return i_str
    

In [116]:
pub_key = PrivateKey(int.from_bytes(master_priv_key,"big")).point.sec()

In [117]:
msg=("0" + hex(int.from_bytes(pub_key,"big"))[2:] + str_i(0)).encode() 
msg

b'0241adae83bb61ff28aaf29a09773b69fec5bcd4a49fbaadfdafbe86a5b031436000000000'

In [118]:
I= hmac.new(
            key = hex(int.from_bytes(chain_code,"big"))[2:].encode() ,
            msg=msg ,
            digestmod=sha512).digest()


In [119]:
I_L, I_R = I[:32], I[32:]

In [120]:
child_privkey_int = (int.from_bytes(I_L,"big") + int.from_bytes(master_priv_key,"big"))%N

In [121]:
child_pubkey_int = (int.from_bytes(I_L,"big") + int.from_bytes(pub_key,"big"))%N

In [122]:
child_privkey = PrivateKey(child_privkey_int).wif(compressed= True)

In [123]:
child_chain_code = I_R

In [124]:
child_privkey

'L29Rc5PyDkbUr2z1VcL2DuAhTQkNo3qwF7ENgRotS86ADUkiXPJF'

In [125]:
hex(int.from_bytes(child_chain_code,"big"))

'0xb3a05798ada17da7a4a7571b47f84a06ef9a0296c46975f455d8d4e37a7386b5'

In [126]:
hex(int.from_bytes(PrivateKey(child_privkey_int).point.sec(),"big"))

'0x2f3dde9a80dbbe0d32f99b87e4eb21f0e332035ae8d9b210e58465c60eef1eabe'

In [127]:
hex(child_pubkey_int)

'0xe2945e1e15ce4e4985777261ad6b1b96519d3e08245071f238d6b5696cf8940a'